<a href="https://colab.research.google.com/github/therealfortune/Data-Engineering/blob/master/ETL_with_different_headers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EXTRACT TRANSFORM LOAD

In [1]:
!unzip DiffSource.zip

Archive:  DiffSource.zip
   creating: DiffSource/
  inflating: DiffSource/source1.csv  
  inflating: DiffSource/source1.json  
  inflating: DiffSource/source1.xml  
  inflating: DiffSource/source2.csv  
  inflating: DiffSource/source2.json  
  inflating: DiffSource/source2.xml  
  inflating: DiffSource/source3.csv  
  inflating: DiffSource/source3.json  
  inflating: DiffSource/source3.xml  


In [5]:
import pandas as pd 
import glob
from datetime import datetime 

In [46]:
a = (pd.read_csv('source1.csv'))
b = pd.read_json("source2.json", lines=True)
c = (pd.read_xml('source1.xml'))
print(a.head(1))
print(b.head(1))
print(c.head(1))

   height  weight  name
0   65.78  112.99  alex
   name  height  weight
0  jack    68.7   123.3
    name  height  weight
0  simon    67.9  112.37


In [20]:
logfile = "/content/log.txt"
tempfile = "/content/temp.tmp"
targetfile ="/content/target.csv"

WE SEE THAT THE CSV FILE HEADERS IS QUITE DIFFERENT FROM THE REST

In [47]:
def extract_csv(file):
    df = pd.read_csv(file)
    df = df[["name", "height", "weight"]]   # Rearraging the csv files
    return df

def extract_json(file):
    df = pd.read_json(file, lines=True)
    return df

def extract_xml(file):
    df = pd.read_xml(file)
    return df

In [44]:
def extract():
    extracted_data = pd.DataFrame(columns = ["name", "height", "weight"])

    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_csv(csvfile), ignore_index = True)

    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_json(jsonfile), ignore_index = True)

    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_xml(xmlfile), ignore_index = True)

    return extracted_data

I organised the csv file headers to look more like the xml and json file headers so that we can easily combine them into one data source which is extracted_data

In [62]:
extracted_data = extract()
extracted_data.head(2)

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49


Transformation Stage

Convert imperial to metrics 

imperial = feet, pounds   metrics = meters , grams

In [63]:
def transform(data):
    # Convert inches into meters
    data["height"] = round(data.height * 0.0254, 2)

    # Converts pounds to kilograms 
    data['weight'] = round(data.weight * 0.45359237, 2)

    return data

In [64]:
transformed = transform(extracted_data)

In [65]:
transformed.head(2)

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91


LOAD --> L

LET US LOAD THE DATA INTO THE TARGET FILE

In [66]:
def load(target, source) :
     source.to_csv(target)

In [69]:
def log(message) :
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timenow = now.strftime(timestamp_format)

    with open("/content/log.txt", "a") as f:
        f.write(timenow + ": " + message + "\n")

In [70]:
log("ETL has started , Be scared !!!")

In [71]:
log("Extract phase has begun")
extracted_data = extract()
log("Extract phase has Ended")
extracted_data

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30
5,alex,65.78,112.99
6,ajay,71.52,136.49
7,alice,69.40,153.03
8,ravi,68.22,142.34
9,joe,67.79,144.30


In [72]:
log("Transform phase has begun")
transformed_data = transform(extracted_data)
log("Transform stage has ended")

In [73]:
transformed_data

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
5,alex,1.67,51.25
6,ajay,1.82,61.91
7,alice,1.76,69.41
8,ravi,1.73,64.56
9,joe,1.72,65.45


In [74]:
log("Load phase has begun")
load(targetfile, transformed_data)
log("Load phase has ended")

In [75]:
log("ETL has been completed . Wheww!!")